In [3]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import trainer
from libs import freeze

In [4]:
flags=tf.app.flags

In [5]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','bed,bird,cat,dog,down,eight,five,four,go,happy,house,left,marvin,nine,no,off,on,one,right,seven,sheila,six,stop,three,tree,two,up,wow,yes,zero','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',30,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',10.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [6]:
model_architecture='conv'
start_checkpoint=None
logging_interval=1
eval_step_interval=1000
save_step_interval=1
silence_percentage=10.0
unknown_percentage=10.0
background_frequency=0.8
background_volume=0.1
learning_rate='0.001,0.0001' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='1000,1000' #Declare  the training steps for which the learning rates will be used
batch_size=100

In [7]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_data, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings, background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_data, train_ground_truth

def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_data, validation_ground_truth = (
        audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_data,validation_ground_truth

In [8]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [9]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [10]:
# WORD_LIST = [x[0].rsplit('\\', 2)[-1] for x in os.walk(train_dir)]
# ','.join(WORD_LIST)

In [11]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings)

..\..\_inputs\raw\train\audio\*\*.wav


W0119 10:56:38.805995 14424 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0119 10:56:38.815957 14424 deprecation_wrapper.py:119] From ..\libs\input_data.py:306: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 98, 257), dtype=float32)


In [12]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    data_size = model_settings['data_size']
    label_count = model_settings['label_count']
    data_input = tf.placeholder(
      tf.float32, [None, data_size], name='data_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    # Create Model
    
    logits, dropout_prob = models.create_model(
      data_input,
      model_settings,
      model_architecture,
      is_training=True)
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,data_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [13]:
tf.app.run(main=main)

W0119 10:57:23.956477 14424 deprecation_wrapper.py:119] From ..\libs\models.py:327: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0119 10:57:23.970406 14424 deprecation.py:506] From ..\libs\models.py:335: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0119 10:57:23.981406 14424 deprecation_wrapper.py:119] From ..\libs\models.py:338: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0119 10:57:24.017329 14424 deprecation.py:323] From ..\libs\trainer.py:57: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.n

I0119 10:58:50.748017 14424 trainer.py:126] Step #26: rate 0.001000, accuracy 19.0%, cross entropy 3.184468
I0119 10:58:50.749958 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-26"
I0119 10:58:54.212543 14424 trainer.py:126] Step #27: rate 0.001000, accuracy 12.0%, cross entropy 3.196301
I0119 10:58:54.213540 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-27"
I0119 10:58:57.689289 14424 trainer.py:126] Step #28: rate 0.001000, accuracy 13.0%, cross entropy 3.226453
I0119 10:58:57.689289 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-28"
I0119 10:59:00.969651 14424 trainer.py:126] Step #29: rate 0.001000, accuracy 14.0%, cross entropy 3.202778
I0119 10:59:00.970650 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-29"
I0119 10:59:04.743645 14424 trainer.py:126] Step #30: rate 0.001000, accuracy 11.0%, cross entropy 3.213243
I0119 10:59:04.744643 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-30"
I0119

I0119 11:01:03.734325 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-67"
I0119 11:01:06.851290 14424 trainer.py:126] Step #68: rate 0.001000, accuracy 27.0%, cross entropy 2.613158
I0119 11:01:06.853285 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-68"
I0119 11:01:09.808871 14424 trainer.py:126] Step #69: rate 0.001000, accuracy 25.0%, cross entropy 2.596534
I0119 11:01:09.809870 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-69"
I0119 11:01:12.915540 14424 trainer.py:126] Step #70: rate 0.001000, accuracy 22.0%, cross entropy 2.822723
I0119 11:01:12.916537 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-70"
I0119 11:01:16.026492 14424 trainer.py:126] Step #71: rate 0.001000, accuracy 30.0%, cross entropy 2.532102
I0119 11:01:16.027491 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-71"
I0119 11:01:19.245698 14424 trainer.py:126] Step #72: rate 0.001000, accuracy 15.0%, cross entropy 2.986165
I0119

I0119 11:03:22.805454 14424 trainer.py:126] Step #109: rate 0.001000, accuracy 26.0%, cross entropy 2.593027
I0119 11:03:22.806436 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-109"
I0119 11:03:25.739315 14424 trainer.py:126] Step #110: rate 0.001000, accuracy 30.0%, cross entropy 2.544605
I0119 11:03:25.740311 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-110"
I0119 11:03:28.949360 14424 trainer.py:126] Step #111: rate 0.001000, accuracy 18.0%, cross entropy 2.723275
I0119 11:03:29.041117 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-111"
I0119 11:03:32.095586 14424 trainer.py:126] Step #112: rate 0.001000, accuracy 36.0%, cross entropy 2.505301
I0119 11:03:32.096530 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-112"
I0119 11:03:35.933178 14424 trainer.py:126] Step #113: rate 0.001000, accuracy 30.0%, cross entropy 2.367467
I0119 11:03:35.934138 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-

I0119 11:06:27.600577 14424 trainer.py:126] Step #150: rate 0.001000, accuracy 38.0%, cross entropy 2.260684
I0119 11:06:27.602644 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-150"
I0119 11:06:33.462520 14424 trainer.py:126] Step #151: rate 0.001000, accuracy 33.0%, cross entropy 2.281903
I0119 11:06:33.465512 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-151"
I0119 11:06:39.190759 14424 trainer.py:126] Step #152: rate 0.001000, accuracy 39.0%, cross entropy 2.159897
I0119 11:06:39.192756 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-152"
I0119 11:06:45.692353 14424 trainer.py:126] Step #153: rate 0.001000, accuracy 42.0%, cross entropy 2.245155
I0119 11:06:45.697367 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-153"
I0119 11:06:51.685569 14424 trainer.py:126] Step #154: rate 0.001000, accuracy 38.0%, cross entropy 2.081847
I0119 11:06:51.687566 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-

...

I0119 13:09:23.714899 14424 trainer.py:126] Step #1414: rate 0.001000, accuracy 63.0%, cross entropy 1.359691
I0119 13:09:23.720882 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1414"
I0119 13:09:29.001325 14424 trainer.py:126] Step #1415: rate 0.001000, accuracy 69.0%, cross entropy 1.078955
I0119 13:09:29.003318 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1415"
I0119 13:09:33.519782 14424 trainer.py:126] Step #1416: rate 0.001000, accuracy 70.0%, cross entropy 1.140820
I0119 13:09:33.521785 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1416"
I0119 13:09:39.401438 14424 trainer.py:126] Step #1417: rate 0.001000, accuracy 65.0%, cross entropy 1.149729
I0119 13:09:39.403442 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1417"
I0119 13:09:43.961277 14424 trainer.py:126] Step #1418: rate 0.001000, accuracy 64.0%, cross entropy 1.108738
I0119 13:09:43.962274 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\c

I0119 13:12:49.828051 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1454"
I0119 13:12:54.329697 14424 trainer.py:126] Step #1455: rate 0.001000, accuracy 72.0%, cross entropy 1.006896
I0119 13:12:54.331692 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1455"
I0119 13:13:00.459059 14424 trainer.py:126] Step #1456: rate 0.001000, accuracy 68.0%, cross entropy 1.189656
I0119 13:13:00.464043 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1456"
I0119 13:13:04.867081 14424 trainer.py:126] Step #1457: rate 0.001000, accuracy 75.0%, cross entropy 0.930261
I0119 13:13:04.870073 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1457"
I0119 13:13:11.258561 14424 trainer.py:126] Step #1458: rate 0.001000, accuracy 76.0%, cross entropy 0.957566
I0119 13:13:11.260557 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1458"
I0119 13:13:16.517592 14424 trainer.py:126] Step #1459: rate 0.001000, accuracy 69.0%, cross en

I0119 13:16:21.658213 14424 trainer.py:126] Step #1495: rate 0.001000, accuracy 66.0%, cross entropy 1.193788
I0119 13:16:21.660209 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1495"
I0119 13:16:25.881701 14424 trainer.py:126] Step #1496: rate 0.001000, accuracy 73.0%, cross entropy 1.050418
I0119 13:16:25.883694 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1496"
I0119 13:16:31.143823 14424 trainer.py:126] Step #1497: rate 0.001000, accuracy 79.0%, cross entropy 0.947829
I0119 13:16:31.145819 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1497"
I0119 13:16:35.595224 14424 trainer.py:126] Step #1498: rate 0.001000, accuracy 73.0%, cross entropy 0.948074
I0119 13:16:35.598214 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1498"
I0119 13:16:41.066574 14424 trainer.py:126] Step #1499: rate 0.001000, accuracy 76.0%, cross entropy 0.964514
I0119 13:16:41.068567 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\c

I0119 13:20:41.248439 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1535"
I0119 13:20:52.123869 14424 trainer.py:126] Step #1536: rate 0.001000, accuracy 64.0%, cross entropy 1.169244
I0119 13:20:52.127884 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1536"
I0119 13:20:58.892719 14424 trainer.py:126] Step #1537: rate 0.001000, accuracy 77.0%, cross entropy 0.895883
I0119 13:20:58.896712 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1537"
I0119 13:21:07.162925 14424 trainer.py:126] Step #1538: rate 0.001000, accuracy 71.0%, cross entropy 0.846767
I0119 13:21:07.184310 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1538"
I0119 13:21:14.700505 14424 trainer.py:126] Step #1539: rate 0.001000, accuracy 74.0%, cross entropy 0.955121
I0119 13:21:14.705572 14424 trainer.py:172] Saving to "..\..\logs&checkpoint\conv\ckpt-1539"
I0119 13:21:21.426465 14424 trainer.py:126] Step #1540: rate 0.001000, accuracy 71.0%, cross en

KeyboardInterrupt: 

In [ ]:
# save_path=os.path.join(FLAGS.models_dir, model_architecture, '%s.pb'%os.path.basename(FLAGS.train_dir))
# freeze.freeze_graph(FLAGS,model_architecture,FLAGS.train_dir,save_path)